In [59]:
import gzip


def unzip(input, output):

    input_file = input
    output_file = output

    with gzip.open(input_file, "rb") as f_in:
        with open(output_file, "wb") as f_out:

            f_out.write(f_in.read())

    print("file unzipped and save at", output_file)

unzip("/data/LSY/z_preparing_and_parts/commontrait_Parkinson/GSE72776_datBetaNormalized.csv.gz",
      "/data/LSY/z_preparing_and_parts/commontrait_Parkinson/GSE72776_series_matrix.txt")

file unzipped and save at /data/LSY/z_preparing_and_parts/commontrait_Parkinson/GSE72776_series_matrix.txt


In [1]:

import pandas as pd



# read matrix
with open(
    "/data/LSY/z_preparing_and_parts/commontrait_Psoriasis/GSE115797_series_matrix.txt", "r"
) as file:
    lines = file.readlines()

# find metadata
first_empty_line_index = lines.index("\n")
table_begin_index = next(
(
        i
        for i, line in enumerate(lines)
        if line.startswith("!series_matrix_table_begin")
    ),
    None,
)
end_index = table_begin_index if table_begin_index is not None else len(lines)
content_before_table = lines[first_empty_line_index:end_index]


# read metadata
metadata_lines = [
    line.strip()[1:].split("\t")
    for line in content_before_table
    if line.startswith("!")
]
metadata_df = pd.DataFrame(metadata_lines).T


metadata_df.columns = metadata_df.iloc[0]
metadata_df = metadata_df.drop(0, axis=0)

metadata_df.dropna(inplace=True)


In [41]:
# mask = metadata_df.iloc[:, 7] == '"Peripheral blood"'

# metadata_df = metadata_df[mask]



In [2]:
print(metadata_df.iloc[:,13].unique())

['"sample type: Adjacent Normal"' '"sample type: Psoriasis lesion"']


In [3]:

# find status
metadata_df["status"] = metadata_df.iloc[:, 13].apply(
    lambda x: 1 if "Psoriasis" in x else 0 if "Normal" in x else None
)
metadata_df = metadata_df.set_index("Sample_geo_accession")

In [4]:

metadata_df = metadata_df["status"].apply(pd.to_numeric, errors="coerce")

In [20]:
print(metadata_df)

Sample_geo_accession
"GSM2844230"    1
"GSM2844231"    1
"GSM2844232"    1
"GSM2844233"    1
"GSM2844234"    1
               ..
"GSM2844550"    0
"GSM2844551"    1
"GSM2844552"    1
"GSM2844553"    0
"GSM2844554"    0
Name: status, Length: 279, dtype: int64


In [5]:


# read data
data_lines = [line.strip().split() for line in lines if not line.startswith("!")]
data_df = pd.DataFrame(data_lines).T

data_df.columns = data_df.iloc[0]
data_df = data_df.drop(0, axis=0)

data_df = data_df.set_index('"ID_REF"')
data_df = data_df.iloc[:, 1:]
data_df.columns = [col.strip('"') for col in data_df.columns]

data_df = data_df.apply(pd.to_numeric, errors="coerce")





In [53]:
print(data_df)

              cg00000029  cg00000108  cg00000109  cg00000165  cg00000236  \
"ID_REF"                                                                   
"GSM1075838"    0.397935    0.988118    0.927119    0.189417    0.808465   
"GSM1075839"    0.580355    0.982527    0.965207    0.180387    0.818062   
"GSM1075840"    0.465099    0.990774    0.915788    0.131453    0.789416   
"GSM1075841"    0.427220    0.988653    0.916545    0.184444    0.612363   
"GSM1075842"    0.421505    0.987646    0.976045    0.170827    0.722557   
...                  ...         ...         ...         ...         ...   
"GSM1075928"    0.472226    0.987939    0.884916    0.201347    0.860997   
"GSM1075929"    0.689183    0.985456    0.811946    0.236332    0.835378   
"GSM1075930"    0.601118    0.988807    0.965625    0.198946    0.784270   
"GSM1075931"    0.486073    0.983242    0.959287    0.137911    0.614809   
"GSM1075932"    0.401654    0.978852    0.912453    0.172783    0.709579   

           

In [9]:

def m_to_beta(M):
    return 1 / (1 + 2 ** (-M))


In [10]:
# data_df = data_df.apply(m_to_beta)

In [6]:

# merge metadata and data
data_df["status"] = metadata_df
# drop other disease sample
data_df.dropna(subset=["status"], inplace=True)

In [24]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_df, test_size=0.2, random_state=42)


# save data
train_data.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Multisclerosis/data_train.txt",
    index=True,
    header=True,
    sep="\t",
)


# save data
test_data.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Multisclerosis/data_test.txt",
    index=True,
    header=True,
    sep="\t",
)


In [7]:

# save data
data_df.to_csv(
    "/data/LSY/z_preparing_and_parts/commontrait_Psoriasis/data_valid.txt",
    index=True,
    header=True,
    sep="\t",
)
